In [8]:
!pip install pandas
!pip install pymysql sqlalchemy
!pip install psycopg2-binary

In [1]:
import pandas as pd

north_file_path = "./data/Pos_north_vn_sales_data.csv"
south_file_path = "./data/erp_south_sales_data_record.csv"

df_north = pd.read_csv(north_file_path)
df_south = pd.read_csv(south_file_path)

df_north["location"] = "north"
df_south["location"] = "south"

df_north.head(), df_south.head()

(                YNSM_NO   YNSM_CUST             ynsd_prod  ynsd_qty  \
 0  S2530030              A0566       PS05848DG54B046B1G01         1   
 1  S2530030              A0566       PS05848DH68B046B1G01         1   
 2  S2530030              A0566       PS05848DLH4B046B1G01         2   
 3  S2530030              A0566       PS05848DH48B04519G01         2   
 4  S2530030              A0566       PS05848DH48B0451BG01         2   
 
     ynsd_amt    YN_DATE location  
 0  477727.27  3/12/2025    north  
 1  477727.27  3/12/2025    north  
 2  477727.27  3/12/2025    north  
 3  477727.27  3/12/2025    north  
 4  477727.27  3/12/2025    north  ,
           PS_NO         dd cus_no cur_id                prd_no  qty  \
 0  SA250312F012  3/12/2025  A0379    NaN  PS05310DB07000000F01   80   
 1  SA250312F014  3/12/2025  A0379    NaN     SXV02529SCA001001  300   
 2  SA250312F018  3/12/2025  A0379    NaN     SXV02529SCA003001  200   
 3  SA250312F015  3/12/2025  A0379    NaN     SXV02529SCA0010

In [2]:
df_north.rename(columns={"YNSM_NO": "order_no", "YNSM_CUST": "customer_id",
                         "ynsd_prod": "product_id", "ynsd_qty": "quantity",
                         "ynsd_amt": "amount", "YN_DATE": "date"}, inplace=True)

df_south.rename(columns={"PS_NO": "order_no", "cus_no": "customer_id",
                         "prd_no": "product_id", "qty": "quantity",
                         "amt": "amount", "dd": "date"}, inplace=True)

df_north["date"] = pd.to_datetime(df_north["date"], errors='coerce')
df_south["date"] = pd.to_datetime(df_south["date"], errors='coerce')
df_south["cur_id"].fillna("VND", inplace=True)

df_north.head(), df_south.head()

(               order_no customer_id            product_id  quantity  \
 0  S2530030              A0566       PS05848DG54B046B1G01         1   
 1  S2530030              A0566       PS05848DH68B046B1G01         1   
 2  S2530030              A0566       PS05848DLH4B046B1G01         2   
 3  S2530030              A0566       PS05848DH48B04519G01         2   
 4  S2530030              A0566       PS05848DH48B0451BG01         2   
 
       amount       date location  
 0  477727.27 2025-03-12    north  
 1  477727.27 2025-03-12    north  
 2  477727.27 2025-03-12    north  
 3  477727.27 2025-03-12    north  
 4  477727.27 2025-03-12    north  ,
        order_no       date customer_id cur_id            product_id  quantity  \
 0  SA250312F012 2025-03-12       A0379    VND  PS05310DB07000000F01        80   
 1  SA250312F014 2025-03-12       A0379    VND     SXV02529SCA001001       300   
 2  SA250312F018 2025-03-12       A0379    VND     SXV02529SCA003001       200   
 3  SA250312F015 2025

In [3]:
import pandas as pd

exchange_rate_file_path = "./data/USD_VND_HistoricalData.csv"

df_exchange = pd.read_csv(exchange_rate_file_path)

df_exchange["Date"] = pd.to_datetime(df_exchange["Date"], errors='coerce')

df_exchange.rename(columns={"Date": "exchange_date", "Price": "exchange_rate"}, inplace=True)

df_exchange = df_exchange.sort_values(by="exchange_date")

date_range = pd.date_range(start=df_exchange["exchange_date"].min(), end=df_exchange["exchange_date"].max(), freq="D")

df_full = pd.DataFrame(date_range, columns=["exchange_date"])

df_merged = df_full.merge(df_exchange, on="exchange_date", how="left")

df_merged["exchange_rate"] = df_merged["exchange_rate"].ffill()

print(df_merged.tail(20))


     exchange_date exchange_rate      Open      High       Low Vol. Change %
1886    2025-03-01      25,530.0       NaN       NaN       NaN  NaN      NaN
1887    2025-03-02      25,530.0       NaN       NaN       NaN  NaN      NaN
1888    2025-03-03      25,575.0  25,554.0  25,617.5  25,548.0  NaN    0.18%
1889    2025-03-04      25,540.0  25,580.0  25,596.5  25,555.0  NaN   -0.14%
1890    2025-03-05      25,490.0  25,515.0  25,586.0  25,495.0  NaN   -0.20%
1891    2025-03-06      25,485.0  25,450.0  25,516.5  25,430.0  NaN   -0.02%
1892    2025-03-07      25,480.0  25,449.0  25,552.5  25,435.0  NaN   -0.02%
1893    2025-03-08      25,480.0       NaN       NaN       NaN  NaN      NaN
1894    2025-03-09      25,480.0       NaN       NaN       NaN  NaN      NaN
1895    2025-03-10      25,525.0  25,515.0  25,568.5  25,510.0  NaN    0.18%
1896    2025-03-11      25,450.0  25,512.5  25,520.0  25,462.0  NaN   -0.29%
1897    2025-03-12      25,445.0  25,485.0  25,515.0  25,450.0  NaN   -0.02%

In [4]:
df_south = df_south.merge(df_merged[["exchange_date", "exchange_rate"]], left_on="date", right_on="exchange_date", how="left")

df_south.drop(columns=["exchange_date"], inplace=True)

df_south["exchange_rate"] = df_south["exchange_rate"].fillna("0")
df_south["exchange_rate"] = df_south["exchange_rate"].str.replace(",", "").astype(float)

df_south["amount_vnd"] = df_south.apply(
    lambda row: row["amount"] if row["cur_id"] == "VND" else row["amount"] * row["exchange_rate"], axis=1
)

df_south.head()

,order_no,date,customer_id,cur_id,product_id,quantity,amount,location,exchange_rate,amount_vnd
0,SA250312F012,2025-03-12,A0379,VND,PS05310DB07000000F01,80,22581818.0,south,25445.0,22581818.0
1,SA250312F014,2025-03-12,A0379,VND,SXV02529SCA001001,300,9081818.0,south,25445.0,9081818.0
2,SA250312F018,2025-03-12,A0379,VND,SXV02529SCA003001,200,6054545.0,south,25445.0,6054545.0
3,SA250312F015,2025-03-12,A0379,VND,SXV02529SCA001001,200,6054545.0,south,25445.0,6054545.0
4,SA250312F016,2025-03-12,A0379,VND,SXV02529SCA001001,200,6054545.0,south,25445.0,6054545.0


In [5]:
df_north = df_north.merge(df_merged[["exchange_date", "exchange_rate"]], left_on="date", right_on="exchange_date", how="left")

df_north.drop(columns=["exchange_date"], inplace=True)

df_south["exchange_rate"] = df_south["exchange_rate"].fillna("0")
df_north["exchange_rate"] = df_north["exchange_rate"].str.replace(",", "").astype(float)

df_north["cur_id"] = "VND"

df_north["amount_vnd"] = df_north.apply(
    lambda row: row["amount"] if row["cur_id"] == "VND" else row["amount"] * row["exchange_rate"], axis=1
)

df_north.head()

,order_no,customer_id,product_id,quantity,amount,date,location,exchange_rate,cur_id,amount_vnd
0,S2530030,A0566,PS05848DG54B046B1G01,1,477727.27,2025-03-12,north,25445.0,VND,477727.27
1,S2530030,A0566,PS05848DH68B046B1G01,1,477727.27,2025-03-12,north,25445.0,VND,477727.27
2,S2530030,A0566,PS05848DLH4B046B1G01,2,477727.27,2025-03-12,north,25445.0,VND,477727.27
3,S2530030,A0566,PS05848DH48B04519G01,2,477727.27,2025-03-12,north,25445.0,VND,477727.27
4,S2530030,A0566,PS05848DH48B0451BG01,2,477727.27,2025-03-12,north,25445.0,VND,477727.27


In [6]:
from sqlalchemy import create_engine, text
from urllib.parse import quote_plus

DB_USER = "dw_user"
DB_PASSWORD = quote_plus("dw_dss_BK@242")
DB_HOST = "localhost"
DB_PORT = "5433"
DB_NAME = "sales_data_warehouse"

engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

df_all = pd.concat([df_north, df_south], ignore_index=True)

#----------------------------------


In [7]:

df_all['date'] = pd.to_datetime(df_all['date']).dt.date 
df_all['location'] = df_all['location'].str.strip() 
df_all['customer_id'] = df_all['customer_id'].str.strip()
df_all['product_id'] = df_all['product_id'].str.strip()
df_all['cur_id'] = df_all['cur_id'].str.strip().str.upper() 


try:
    with engine.connect() as connection:
        with connection.begin():
            print("Transaction started.")

            # === 1. Load DIM_AREA ===
            print("Loading dim_area...")
            unique_locations = df_all[['location']].drop_duplicates().dropna()
            if not unique_locations.empty:
                area_data = unique_locations.to_dict(orient='records')
                stmt_area = text("""
                    INSERT INTO dim_area (area_name)
                    VALUES (:location)
                    ON CONFLICT (area_name) DO NOTHING;
                """)
                connection.execute(stmt_area, area_data)
                print(f"Processed {len(area_data)} unique areas.")

            # Lấy mapping area_name -> area_id
            area_map_result = connection.execute(text("SELECT area_name, area_id FROM dim_area;"))
            area_map = {row.area_name: row.area_id for row in area_map_result}
            print("Area map created.")

            # === 2. Load DIM_DATE ===
            print("Loading dim_date...")
            unique_dates_df = df_all[['date']].drop_duplicates().dropna()
            if not unique_dates_df.empty:
                date_data = []
                for date_val in unique_dates_df['date']:
                    date_data.append({
                        'date': date_val,
                        'year': date_val.year,
                        'month': date_val.month,
                        'day': date_val.day,
                        'quarter': (date_val.month - 1) // 3 + 1
                    })

                stmt_date = text("""
                    INSERT INTO dim_date (date, year, month, day, quarter)
                    VALUES (:date, :year, :month, :day, :quarter)
                    ON CONFLICT (date) DO NOTHING;
                """)
                connection.execute(stmt_date, date_data)
                print(f"Processed {len(date_data)} unique dates.")

            # Lấy mapping date -> date_id
            date_map_result = connection.execute(text("SELECT date, date_id FROM dim_date;"))
            date_map = {row.date: row.date_id for row in date_map_result}
            print("Date map created.")


            # === 3. Load DIM_CURRENCY ===
            print("Loading dim_currency...")
            unique_currencies = df_all[['cur_id']].drop_duplicates().dropna()
            if not unique_currencies.empty:
                currency_data = unique_currencies.rename(columns={'cur_id': 'cur_code'}).to_dict(orient='records')
                stmt_currency = text("""
                    INSERT INTO dim_currency (cur_code)
                    VALUES (:cur_code)
                    ON CONFLICT (cur_code) DO NOTHING;
                """)
                connection.execute(stmt_currency, currency_data)
                print(f"Processed {len(currency_data)} unique currencies.")

            # Lấy mapping cur_code -> currency_id
            currency_map_result = connection.execute(text("SELECT cur_code, currency_id FROM dim_currency;"))
            currency_map = {row.cur_code: row.currency_id for row in currency_map_result}
            print("Currency map created.")


            # === 4. Load DIM_CUSTOMER ===
            print("Loading dim_customer...")
            unique_customers = df_all[['customer_id', 'location']].drop_duplicates().dropna(subset=['customer_id'])
            if not unique_customers.empty:
                customer_data = []
                for _, row in unique_customers.iterrows():
                    area_id = area_map.get(row['location'])
                    if area_id is None:
                        print(f"Warning: Skipping customer {row['customer_id']} due to unknown location '{row['location']}'.")
                        continue
                    customer_data.append({
                        'cus_no': row['customer_id'],
                        'customer_name': row['customer_id'],
                        'area_id': area_id
                    })

                if customer_data: 
                    stmt_customer = text("""
                        INSERT INTO dim_customer (cus_no, customer_name, area_id)
                        VALUES (:cus_no, :customer_name, :area_id)
                        ON CONFLICT (cus_no) DO NOTHING;
                    """)
                    connection.execute(stmt_customer, customer_data)
                    print(f"Processed {len(customer_data)} unique customers.")

            # Lấy mapping cus_no -> customer_id (PK của bảng)
            customer_map_result = connection.execute(text("SELECT cus_no, customer_id FROM dim_customer;"))
            customer_map = {row.cus_no: row.customer_id for row in customer_map_result}
            print("Customer map created.")


            # === 5. Load DIM_PRODUCT ===
            print("Loading dim_product...")
            unique_products = df_all[['product_id']].drop_duplicates().dropna()
            if not unique_products.empty:
                product_data = []
                for _, row in unique_products.iterrows():
                    # Giả định tên = mã SP và các trường khác là NULL
                    product_data.append({
                        'prd_no': row['product_id'],
                        'prd_name': row['product_id'], # Giả sử tên = mã SP
                        'logo': None, 'model': None, 'std': None, 'color': None,
                        'sqa': None, 'sqa_color': None, 'size_': None
                    })

                stmt_product = text("""
                    INSERT INTO dim_product
                    (prd_no, prd_name, logo, model, std, color, sqa, sqa_color, size_)
                    VALUES
                    (:prd_no, :prd_name, :logo, :model, :std, :color, :sqa, :sqa_color, :size_)
                    ON CONFLICT (prd_no) DO NOTHING;
                """)
                connection.execute(stmt_product, product_data)
                print(f"Processed {len(product_data)} unique products.")

            # Lấy mapping prd_no -> product_id (PK của bảng)
            product_map_result = connection.execute(text("SELECT prd_no, product_id FROM dim_product;"))
            product_map = {row.prd_no: row.product_id for row in product_map_result}
            print("Product map created.")


            # === 6. Load FACT_SALES ===
            print("Loading fact_sales...")
            fact_data = []
            processed_count = 0
            skipped_count = 0

            for index, row in df_all.iterrows():
                # Lookup foreign keys
                date_id = date_map.get(row['date'])
                customer_id_fk = customer_map.get(row['customer_id'])
                currency_id_fk = currency_map.get(row['cur_id'])
                product_id_fk = product_map.get(row['product_id'])

                # Validate foreign keys
                if None in [date_id, customer_id_fk, currency_id_fk, product_id_fk]:
                    skipped_count += 1
                    continue


                try:
                    qty = int(row['quantity'])
                    amt = float(row['amount'])
                    amt_vnd = float(row['amount_vnd'])
                    exchange_rate = float(row['exchange_rate'])
                    if qty <= 0 or amt < 0 or exchange_rate <= 0:
                         skipped_count += 1
                         continue
                except (ValueError, TypeError):
                    skipped_count += 1
                    continue


                fact_data.append({
                    'ps_no': row['order_no'],
                    'date_id': date_id,
                    'customer_id': customer_id_fk,
                    'currency_id': currency_id_fk,
                    'product_id': product_id_fk,
                    'qty': qty,
                    'amt': amt,
                    'exchange_rate': exchange_rate,
                    'amt_vnd': amt_vnd
                })
                processed_count += 1


            if fact_data:
                stmt_fact = text("""
                    INSERT INTO fact_sales
                    (ps_no, date_id, customer_id, currency_id, product_id, qty, amt, exchange_rate, amt_vnd)
                    VALUES
                    (:ps_no, :date_id, :customer_id, :currency_id, :product_id, :qty, :amt, :exchange_rate, :amt_vnd);
                """)

                connection.execute(stmt_fact, fact_data)
                print(f"Inserted {len(fact_data)} rows into fact_sales.")
            else:
                print("No valid data to insert into fact_sales.")

            if skipped_count > 0:
                 print(f"Total rows skipped due to missing FKs or invalid data: {skipped_count}")

        print("Transaction committed successfully.")

except Exception as e:
    print(f"ETL failed. Transaction rolled back. Error: {e}")

finally:
    print("ETL process finished.")

Transaction started.
Loading dim_area...
Processed 2 unique areas.
Area map created.
Loading dim_date...
Processed 1553 unique dates.
Date map created.
Loading dim_currency...
Processed 2 unique currencies.
Currency map created.
Loading dim_customer...
Processed 672 unique customers.
Customer map created.
Loading dim_product...
Processed 25272 unique products.
Product map created.
Loading fact_sales...
Inserted 212606 rows into fact_sales.
Transaction committed successfully.
ETL process finished.
